In [1]:
def create_from_to(file):
    """
    Creates from to file from the combined excel database of boreholes.
    Accepts excel ONLY.
    """
    df=pd.read_excel(file)
    HOLE_ID = df.HOLE_ID.values # Which BH(s) would you like to process?


In [2]:
#Initializing: import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Initializing: define data checking functions
def check_second_row(df):
    """
    Checks and removes if the sheet has <UNIT> row.
    """
    if df.HOLE_ID[0] == ('<UNITS>' or '<UNIT>'):
        df = df.drop([0])
    return df

def clean_geol_cont(GEOL):
    """
    Checks and cleans if the GEOL sheet contains <CONT> rows.
    """
    i=0    #Resets initial index
    for i in range(len(GEOL)):
        if GEOL.HOLE_ID[i] == '<CONT>':
            GEOL.GEOL_DESC[i-1] = GEOL.GEOL_DESC[i-1]+' '+GEOL.GEOL_DESC[i]
            GEOL.GEOL_LEG[i-1] = GEOL.GEOL_LEG[i]
            GEOL.GEOL_STAT[i-1] = GEOL.GEOL_STAT[i]
            GEOL = GEOL.drop([i])
    return GEOL

def clean_hole_cont(HOLE):
    """
    Checks and cleans if the HOLE sheet contains <CONT> rows.
    """
    i=0    #Resets initial index
    for i in range(len(GEOL)):
        if GEOL.HOLE_ID[i] == '<CONT>':
            HOLE.HOLE_REM[i-1] = HOLE.HOLE_REM[i-1]+' '+HOLE.HOLE_REM[i]
            HOLE.HOLE_ORNT[i-1] = HOLE.HOLE_ORNT[i]
            HOLE.HOLE_INCL[i-1] = HOLE.HOLE_INCL[i]
            HOLE = HOLE.drop([i])
    return HOLE

In [3]:
'''
Step 1.1: Reads different tabs of the excel file created from AGS as different DataFrames
'''

## Optional part for combining survey data for 66917, 66917
#HORN = pd.read_excel('66917-AGS.xlsx', sheet_name='%%dHORN')
#HORN = check_second_row(HORN)


concat_file = 'Concat_data_combined.xlsx'



## Compulsory:
data = {}
for sheet in ['HOLE', 'CORE', 'DETL', 'WETH', 'FRAC', 'GEOL']:
    data[sheet] = pd.read_excel(concat_file, sheet_name=sheet)
    data[sheet]['HOLE_ID'] = np.where(
        data[sheet]['AGS_REP'].str[:5].str.contains('[a-zA-Z]', na=False),
        data[sheet]['HOLE_ID'], # Value if condition is True (contains letters)
        data[sheet]['AGS_REP'].str[:5] + "_" + data[sheet]['HOLE_ID']) # Value if condition is False (no letters)
    sheet = check_second_row(data[sheet])

if '<CONT>' in data['GEOL'].HOLE_ID.values:  
    GEOL = clean_geol_cont(data['GEOL'])

CORE = data['CORE'] 
DETL = data['DETL']
WETH = data['WETH']
FRAC = data['FRAC']
GEOL = data['GEOL']
HOLE = data['HOLE']

#HOLE_ID = HOLE.HOLE_ID.values
filtered = HOLE[HOLE['Include'] >= 1]
HOLE_ID = filtered.HOLE_ID.values

In [4]:
print(len(HOLE_ID))

942


In [6]:
from tqdm import tqdm

'''
Step 2: Creates a master DataFrame, assigns the column headings and put information into it
'''

df=pd.DataFrame(columns=['HOLE_ID','DEPTH_FROM','DEPTH_TO','GEOL','GEOL_DESC','THICKNESS_M','TCR','RQD','NR','Mod_Weak',
                         'Weak','Ext_Weak','WETH_GRAD','FI','Details','Fault','Corestone','Breccia','Soil','Jn','Jr_Max',
                         'Jr_Min','Ja_Max','Ja_Min','Jw','SRF',"Q'",'Q'],index=[0])


row_index = 0                                #Creates an accumulating index for recursive input to dataframe
for bh in tqdm(range(len(HOLE_ID))):
    #=====PART 1: getting combined from/to=====
    #getting a sorted union of all depths
    all_depths=[]
    depth_from=[]
    depth_to=[]
    all_depths.append(list(CORE[CORE['HOLE_ID']==HOLE_ID[bh]]['CORE_TOP'].values))
    all_depths.append(list(CORE[CORE['HOLE_ID']==HOLE_ID[bh]]['CORE_BOT'].values))
    all_depths.append(list(DETL[DETL['HOLE_ID']==HOLE_ID[bh]]['DETL_TOP'].values))
    all_depths.append(list(DETL[DETL['HOLE_ID']==HOLE_ID[bh]]['DETL_BASE'].values))
    all_depths.append(list(WETH[WETH['HOLE_ID']==HOLE_ID[bh]]['WETH_TOP'].values))
    all_depths.append(list(WETH[WETH['HOLE_ID']==HOLE_ID[bh]]['WETH_BASE'].values))
    all_depths.append(list(FRAC[FRAC['HOLE_ID']==HOLE_ID[bh]]['FRAC_TOP'].values))
    all_depths.append(list(FRAC[FRAC['HOLE_ID']==HOLE_ID[bh]]['FRAC_BASE'].values))
    all_depths.append(list(GEOL[GEOL['HOLE_ID']==HOLE_ID[bh]]['GEOL_TOP'].values))
    all_depths.append(list(GEOL[GEOL['HOLE_ID']==HOLE_ID[bh]]['GEOL_BASE'].values))
        
        ## Optional:
    #    all_depths.append(list(HORN[HORN['HOLE_ID']==HOLE_ID[bh]]['HORN_TOP'].values))
    #    all_depths.append(list(HORN[HORN['HOLE_ID']==HOLE_ID[bh]]['HORN_BASE'].values))
    
    depths=list(set().union(*all_depths))
    depths.sort()
    depth = [x for x in depths if str(x) != 'nan']   # Removes NaN values from the sorted depths list
    depth.sort()
    #create a "from and to" from this list
    depth_from=depth[0:-1]
    depth_to=depth[1:]

    #=====PART 2: filling in data into the dataframe=====
    #filling the from/to into the master DF
    d = 0
    for d in range(len(depth_to)):
        df.loc[row_index,'HOLE_ID'] = HOLE_ID[bh]
        df.loc[row_index,'DEPTH_FROM'] = depth_from[d]
        df.loc[row_index,'DEPTH_TO'] = depth_to[d]
        df.loc[row_index,'THICKNESS_M'] = depth_to[d]-depth_from[d]
        row_index=row_index+1
    
    #filling from WETH sheet
    for w in range(0,len(list(WETH[WETH['HOLE_ID']==HOLE_ID[bh]]['WETH_TOP']))):
        weth = WETH.loc[(WETH['HOLE_ID']== HOLE_ID[bh])]
        df.loc[(df.HOLE_ID==HOLE_ID[bh])&(df.DEPTH_FROM.between(weth.iloc[w]['WETH_TOP'],weth.iloc[w]['WETH_BASE']-.001)),
               'WETH_GRAD'] = weth.iloc[w]['WETH_GRAD']
    
    #filling from GEOL sheet
    for g in range(0,len(list(GEOL[GEOL['HOLE_ID']==HOLE_ID[bh]]['GEOL_TOP']))):
        geol = GEOL.loc[(GEOL['HOLE_ID']== HOLE_ID[bh])]
        df.loc[(df.HOLE_ID==HOLE_ID[bh])&(df.DEPTH_FROM.between(geol.iloc[g]['GEOL_TOP'],geol.iloc[g]['GEOL_BASE']-.001)),
               'GEOL'] = geol.iloc[g]['GEOL_LEG']
        df.loc[(df.HOLE_ID==HOLE_ID[bh])&(df.DEPTH_FROM.between(geol.iloc[g]['GEOL_TOP'],geol.iloc[g]['GEOL_BASE']-.001)),
               'GEOL_DESC'] = geol.iloc[g]['GEOL_DESC']
    
    #filling from DETL sheet
    for d in range(0,len(list(DETL[DETL['HOLE_ID']==HOLE_ID[bh]]['DETL_TOP']))):
        detl = DETL.loc[(DETL['HOLE_ID']== HOLE_ID[bh])]
        df.loc[(df.HOLE_ID==HOLE_ID[bh])&(df.DEPTH_FROM.between(detl.iloc[d]['DETL_TOP'],detl.iloc[d]['DETL_BASE']-.001)),
               'Details'] = detl.iloc[d]['DETL_DESC']
    
    #filling from CORE sheet
    for c in range(0,len(list(CORE[CORE['HOLE_ID']==HOLE_ID[bh]]['CORE_TOP']))):
        core = CORE.loc[(CORE['HOLE_ID']== HOLE_ID[bh])]
        df.loc[(df.HOLE_ID==HOLE_ID[bh])&(df.DEPTH_FROM.between(core.iloc[c]['CORE_TOP'],core.iloc[c]['CORE_BOT']-.001)),
               'RQD'] = core.iloc[c]['CORE_RQD']
        df.loc[(df.HOLE_ID==HOLE_ID[bh])&(df.DEPTH_FROM.between(core.iloc[c]['CORE_TOP'],core.iloc[c]['CORE_BOT']-.001)),
               'TCR'] = core.iloc[c]['CORE_PREC']
    
    #filling from FRAC sheet
    for fr in range(1,len(list(FRAC[FRAC['HOLE_ID']==HOLE_ID[bh]]['FRAC_TOP']))):
        frac = FRAC.loc[(FRAC['HOLE_ID']== HOLE_ID[bh])]
        df.loc[(df.HOLE_ID==HOLE_ID[bh])&(df.DEPTH_FROM.between(frac.iloc[fr]['FRAC_TOP'],frac.iloc[fr]['FRAC_BASE']-.001)),
               'FI'] = frac.iloc[fr]['FRAC_FI']

    ## Optional: filling from HORN sheet
#    for h in range(0,len(list(HORN[HORN['HOLE_ID']==HOLE_ID[bh]]['HORN_TOP']))):
#        horn = HORN.loc[(HORN['HOLE_ID']== HOLE_ID[bh])]
#        df.loc[(df.HOLE_ID==HOLE_ID[bh])&(df.DEPTH_FROM.between(horn.iloc[h]['HORN_TOP'],horn.iloc[h]['HORN_BASE']-.001)),
#               'HORN'] = horn.iloc[h]['HORN']

df

100%|██████████| 942/942 [10:46<00:00,  1.46it/s]


,HOLE_ID,DEPTH_FROM,DEPTH_TO,GEOL,GEOL_DESC,THICKNESS_M,TCR,RQD,NR,Mod_Weak,...,Soil,Jn,Jr_Max,Jr_Min,Ja_Max,Ja_Min,Jw,SRF,Q',Q
0,NOL-1631-DH01,0.0,1.0,FILL,"Firm, brown, slightly clayey SILT with occasio...",1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NOL-1631-DH01,1.0,2.5,CLAYZ,"Soft, light brown, slightly silty CLAY (ALLUVIUM)",1.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NOL-1631-DH01,2.5,3.6,SILTCS,"Firm, light brown, slightly clayey SILT with o...",1.1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NOL-1631-DH01,3.6,5.6,SILTC,"Extremely weak, light brown, spotted white, co...",2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NOL-1631-DH01,5.6,7.6,SILTC,"Extremely weak, light brown, spotted white, co...",2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37253,68240_BH 5,34.63,35.11,NaN,"Moderately strong, red spotted black and white...",0.48,96.0,70.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37254,68240_BH 5,35.11,35.54,NaN,"Moderately strong, red spotted black and white...",0.43,100.0,74.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37255,68240_BH 5,35.54,35.69,NaN,"Moderately strong, red spotted black and white...",0.15,100.0,74.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37256,68240_BH 5,35.69,35.96,NaN,"Moderately strong, red spotted black and white...",0.27,100.0,74.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
'''
Step 3: Key info extraction and interpretation
'''

#Step 3.1: Get "weak"s from dataframe
df['Mod_Weak'] = (df['Details'].str.contains('moderately weak', case = False, na = False))|(df['GEOL_DESC'].str.contains(
    'moderately weak', case = False))
df['Mod_Weak'].fillna(False,inplace=True)
df['Weak'] = (df['Details'].str.contains('weak', case = False, na = False))|(df['GEOL_DESC'].str.contains('weak', 
                                                                                                          case = False))
df['Weak'].fillna(False,inplace=True)
df['Ext_Weak'] = (df['Details'].str.contains('extremely weak', case = False, na = False))|(df['GEOL_DESC'].str.contains(
    'extremely weak', case = False))
df['Ext_Weak'].fillna(False,inplace=True)
df['NR'] = (df['Details'].str.contains('no recovery', case = False, na = False))|(df['GEOL_DESC'].str.contains(
    'no recovery',case = False))
df['NR'].fillna(False,inplace=True)

#Catch special words from descriptions
df['Fault'] = (df['Details'].str.contains('fault', case = False, na = False))|(df['GEOL_DESC'].str.contains('fault', 
                                                                                                            case = False))
df['Fault'].fillna(False,inplace=True)
df['Corestone'] = (df['Details'].str.contains('corestone', case = False, na = False))|(df['GEOL_DESC'].str.contains(
    'corestone',case = False))
df['Corestone'].fillna(False,inplace=True)
df['Breccia'] = (df['Details'].str.contains('breccia', case = False, na = False))|(df['GEOL_DESC'].str.contains('breccia', 
                                                                                                                case = False))
df['Breccia'].fillna(False,inplace=True)

#Step 3.2: Search for soil stratum

## Top Soils (clay, fines, silt, sand only)
df.loc[(df['GEOL'].str.contains('CLAY', case = True))&(df['GEOL_DESC'].str.contains('TOPSOIL',
                                                                                    case = True)),'Soil']='TS-c'
df.loc[(df['GEOL'].str.contains('FINE', case = True))&(df['GEOL_DESC'].str.contains('TOPSOIL',
                                                                                    case = True)),'Soil']='TS-c/z'
df.loc[(df['GEOL'].str.contains('SILT', case = True))&(df['GEOL_DESC'].str.contains('TOPSOIL',
                                                                                    case = True)),'Soil']='TS-z'
df.loc[(df['GEOL'].str.contains('SAND', case = True))&(df['GEOL_DESC'].str.contains('TOPSOIL',
                                                                                    case = True)),'Soil']='TS-s'

## Marine Deposit (clay, fines, silt, sand, gravel, cbbl(?))
df.loc[(df['GEOL'].str.contains('CLAY', case = True))&(df['GEOL_DESC'].str.contains('MARINE DEPOSIT',
                                                                                    case = True)),'Soil']='MD-c'
df.loc[(df['GEOL'].str.contains('FINE', case = True))&(df['GEOL_DESC'].str.contains('MARINE DEPOSIT',
                                                                                    case = True)),'Soil']='MD-c/z'
df.loc[(df['GEOL'].str.contains('SILT', case = True))&(df['GEOL_DESC'].str.contains('MARINE DEPOSIT',
                                                                                    case = True)),'Soil']='MD-z'
df.loc[(df['GEOL'].str.contains('SAND', case = True))&(df['GEOL_DESC'].str.contains('MARINE DEPOSIT',
                                                                                    case = True)),'Soil']='MD-s'
df.loc[(df['GEOL'].str.contains('GRAV', case = True))&(df['GEOL_DESC'].str.contains('MARINE DEPOSIT',
                                                                                    case = True)),'Soil']='MD-g'
df.loc[(df['GEOL'].str.contains('CBBL', case = True))&(df['GEOL_DESC'].str.contains('MARINE DEPOSIT',
                                                                                    case = True)),'Soil']='MD-cb'

## Alluvium (clay, fines, silt, sand, gravel, cbbl, boulder)
df.loc[(df['GEOL'].str.contains('CLAY', case = True))&(df['GEOL_DESC'].str.contains('ALLUVIUM',
                                                                                    case = True)),'Soil']='ALL-c'
df.loc[(df['GEOL'].str.contains('FINE', case = True))&(df['GEOL_DESC'].str.contains('ALLUVIUM',
                                                                                    case = True)),'Soil']='ALL-c/z'
df.loc[(df['GEOL'].str.contains('SILT', case = True))&(df['GEOL_DESC'].str.contains('ALLUVIUM',
                                                                                    case = True)),'Soil']='ALL-z'
df.loc[(df['GEOL'].str.contains('SAND', case = True))&(df['GEOL_DESC'].str.contains('ALLUVIUM',
                                                                                    case = True)),'Soil']='ALL-s'
df.loc[(df['GEOL'].str.contains('GRAV', case = True))&(df['GEOL_DESC'].str.contains('ALLUVIUM',
                                                                                    case = True)),'Soil']='ALL-g'
df.loc[(df['GEOL'].str.contains('CBBL', case = True))&(df['GEOL_DESC'].str.contains('ALLUVIUM',
                                                                                    case = True)),'Soil']='ALL-cb'
df.loc[(df['GEOL'].str.contains('BLDR', case = True))&(df['GEOL_DESC'].str.contains('ALLUVIUM',
                                                                                    case = True)),'Soil']='ALL-bd'

## Colluvium (clay, fines, silt, sand, gravel, cbbl,boulder)
df.loc[(df['GEOL'].str.contains('CLAY', case = True))&(df['GEOL_DESC'].str.contains('COLLUVIUM',
                                                                                    case = True)),'Soil']='COLL-c'
df.loc[(df['GEOL'].str.contains('FINE', case = True))&(df['GEOL_DESC'].str.contains('COLLUVIUM',
                                                                                    case = True)),'Soil']='COLL-c/z'
df.loc[(df['GEOL'].str.contains('SILT', case = True))&(df['GEOL_DESC'].str.contains('COLLUVIUM',
                                                                                    case = True)),'Soil']='COLL-z'
df.loc[(df['GEOL'].str.contains('SAND', case = True))&(df['GEOL_DESC'].str.contains('COLLUVIUM',
                                                                                    case = True)),'Soil']='COLL-s'
df.loc[(df['GEOL'].str.contains('GRAV', case = True))&(df['GEOL_DESC'].str.contains('COLLUVIUM',
                                                                                    case = True)),'Soil']='COLL-g'
df.loc[(df['GEOL'].str.contains('CBBL', case = True))&(df['GEOL_DESC'].str.contains('COLLUVIUM',
                                                                                    case = True)),'Soil']='COLL-cb'
df.loc[(df['GEOL'].str.contains('BLDR', case = True))&(df['GEOL_DESC'].str.contains('COLLUVIUM',
                                                                                    case = True)),'Soil']='COLL-bd'

## FILL (clay, silt, sand, gravel, cbbl,boulder)
df.loc[(df['GEOL_DESC'].str.contains('CLAY', case = True))&(df['GEOL_DESC'].str.contains('FILL',
                                                                                    case = True)),'Soil']='FILL-c'
df.loc[(df['GEOL_DESC'].str.contains('SILT', case = True))&(df['GEOL_DESC'].str.contains('FILL',
                                                                                    case = True)),'Soil']='FILL-z'
df.loc[(df['GEOL_DESC'].str.contains('SAND', case = True))&(df['GEOL_DESC'].str.contains('FILL',
                                                                                    case = True)),'Soil']='FILL-s'
df.loc[(df['GEOL_DESC'].str.contains('GRAVEL', case = True))&(df['GEOL_DESC'].str.contains('FILL',
                                                                                    case = True)),'Soil']='FILL-g'
df.loc[(df['GEOL_DESC'].str.contains('COBBLE', case = True))&(df['GEOL_DESC'].str.contains('FILL',
                                                                                    case = True)),'Soil']='FILL-cb'
df.loc[(df['GEOL_DESC'].str.contains('BOULDER', case = True))&(df['GEOL_DESC'].str.contains('FILL',
                                                                                    case = True)),'Soil']='FILL-bd'

## Residual Soil(clay, silt, sand, gravel, cbbl,boulder)
### Check once in GEOL & GEOL_DESC columns
df.loc[(df['WETH_GRAD']=='VI')&((df['GEOL_DESC'].str.contains('CLAY',case = True))|(df['GEOL'].str.contains(
    'CLAY',case = True))),'Soil']='VI-c'
df.loc[(df['WETH_GRAD']=='VI')&((df['GEOL_DESC'].str.contains('SILT',case = True))|(df['GEOL'].str.contains(
    'SILT',case = True))),'Soil']='VI-z'
df.loc[(df['WETH_GRAD']=='VI')&((df['GEOL_DESC'].str.contains('SAND',case = True))|(df['GEOL'].str.contains(
    'SAND',case = True))),'Soil']='VI-s'
df.loc[(df['WETH_GRAD']=='VI')&((df['GEOL_DESC'].str.contains('GRAVEL',case = True))|(df['GEOL'].str.contains(
    'GRAV',case = True))),'Soil']='VI-g'
df.loc[(df['WETH_GRAD']=='VI')&((df['GEOL_DESC'].str.contains('COBBLE',case = True))|(df['GEOL'].str.contains(
    'CBBL',case = True))),'Soil']='VI-cb'
df.loc[(df['WETH_GRAD']=='VI')&((df['GEOL_DESC'].str.contains('BOULDER',case = True))|(df['GEOL'].str.contains(
    'BLDR',case = True))),'Soil']='VI-bd'
### Check once more in Details column
df.loc[(df['WETH_GRAD']=='VI')&((df['Details'].str.contains('CLAY',case = True))),'Soil']='VI-c'
df.loc[(df['WETH_GRAD']=='VI')&((df['Details'].str.contains('SILT',case = True))),'Soil']='VI-z'
df.loc[(df['WETH_GRAD']=='VI')&((df['Details'].str.contains('SAND',case = True))),'Soil']='VI-s'
df.loc[(df['WETH_GRAD']=='VI')&((df['Details'].str.contains('GRAVEL',case = True))),'Soil']='VI-g'
df.loc[(df['WETH_GRAD']=='VI')&((df['Details'].str.contains('COBBLE',case = True))),'Soil']='VI-cb'
df.loc[(df['WETH_GRAD']=='VI')&((df['Details'].str.contains('BOUDLER',case = True))),'Soil']='VI-bl'

## CDG (clay, silt, sand, gravel, cbbl,boulder)
### Check once in GEOL & GEOL_DESC columns
df.loc[(df['WETH_GRAD']=='V')&((df['GEOL_DESC'].str.contains('CLAY',case = True))|(df['GEOL'].str.contains(
    'CLAY',case = True))),'Soil']='V-c'
df.loc[(df['WETH_GRAD']=='V')&((df['GEOL_DESC'].str.contains('SILT',case = True))|(df['GEOL'].str.contains(
    'SILT',case = True))),'Soil']='V-z'
df.loc[(df['WETH_GRAD']=='V')&((df['GEOL_DESC'].str.contains('SAND',case = True))|(df['GEOL'].str.contains(
    'SAND',case = True))),'Soil']='V-s'
df.loc[(df['WETH_GRAD']=='V')&((df['GEOL_DESC'].str.contains('GRAVEL',case = True))|(df['GEOL'].str.contains(
    'GRAV',case = True))),'Soil']='V-g'
df.loc[(df['WETH_GRAD']=='V')&((df['GEOL_DESC'].str.contains('COBBLE',case = True))|(df['GEOL'].str.contains(
    'CBBL',case = True))),'Soil']='V-cb'
df.loc[(df['WETH_GRAD']=='V')&((df['GEOL_DESC'].str.contains('BOULDER',case = True))|(df['GEOL'].str.contains(
    'BLDR',case = True))),'Soil']='V-bd'
### Check once more in Details column
df.loc[(df['WETH_GRAD']=='V')&((df['Details'].str.contains('CLAY',case = True))),'Soil']='V-c'
df.loc[(df['WETH_GRAD']=='V')&((df['Details'].str.contains('SILT',case = True))),'Soil']='V-z'
df.loc[(df['WETH_GRAD']=='V')&((df['Details'].str.contains('SAND',case = True))),'Soil']='V-s'
df.loc[(df['WETH_GRAD']=='V')&((df['Details'].str.contains('GRAVEL',case = True))),'Soil']='V-g'
df.loc[(df['WETH_GRAD']=='V')&((df['Details'].str.contains('COBBLE',case = True))),'Soil']='V-cb'
df.loc[(df['WETH_GRAD']=='V')&((df['Details'].str.contains('BOUDLER',case = True))),'Soil']='V-bl'

## HDG (clay, silt, sand, gravel, cbbl,boulder)
### Check once in GEOL & GEOL_DESC columns
df.loc[(df['WETH_GRAD']=='IV')&((df['GEOL_DESC'].str.contains('CLAY',case = True))|(df['GEOL'].str.contains(
    'CLAY',case = True))),'Soil']='IV-c'
df.loc[(df['WETH_GRAD']=='IV')&((df['GEOL_DESC'].str.contains('SILT',case = True))|(df['GEOL'].str.contains(
    'SILT',case = True))),'Soil']='IV-z'
df.loc[(df['WETH_GRAD']=='IV')&((df['GEOL_DESC'].str.contains('SAND',case = True))|(df['GEOL'].str.contains(
    'SAND',case = True))),'Soil']='IV-s'
df.loc[(df['WETH_GRAD']=='IV')&((df['GEOL_DESC'].str.contains('GRAVEL',case = True))|(df['GEOL'].str.contains(
    'GRAV',case = True))),'Soil']='IV-g'
df.loc[(df['WETH_GRAD']=='IV')&((df['GEOL_DESC'].str.contains('COBBLE',case = True))|(df['GEOL'].str.contains(
    'CBBL',case = True))),'Soil']='IV-cb'
df.loc[(df['WETH_GRAD']=='IV')&((df['GEOL_DESC'].str.contains('BOULDER',case = True))|(df['GEOL'].str.contains(
    'BLDR',case = True))),'Soil']='IV-bd'
### Check once more in Details column
df.loc[(df['WETH_GRAD']=='IV')&((df['Details'].str.contains('CLAY',case = True))),'Soil']='IV-c'
df.loc[(df['WETH_GRAD']=='IV')&((df['Details'].str.contains('SILT',case = True))),'Soil']='IV-z'
df.loc[(df['WETH_GRAD']=='IV')&((df['Details'].str.contains('SAND',case = True))),'Soil']='IV-s'
df.loc[(df['WETH_GRAD']=='IV')&((df['Details'].str.contains('GRAVEL',case = True))),'Soil']='IV-g'
df.loc[(df['WETH_GRAD']=='IV')&((df['Details'].str.contains('COBBLE',case = True))),'Soil']='IV-cb'
df.loc[(df['WETH_GRAD']=='IV')&((df['Details'].str.contains('BOUDLER',case = True))),'Soil']='IV-bl'

C:\Users\regine.tsui\AppData\Local\Temp\13\ipykernel_31884\3487473668.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Mod_Weak'].fillna(False,inplace=True)
C:\Users\regine.tsui\AppData\Local\Temp\13\ipykernel_31884\3487473668.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [8]:
'''
Step 3.3: Calculates Jn, Jr , Ja from RQD and descriptions
'''

# Define condition where Q calculation is valid:
is_rock = (df['RQD'].notna())&(df['RQD']!=0)&(df['NR']==False)&(df['WETH_GRAD'].str.contains('V', case = True)==False)

## Jn (from RQD)
df.loc[(df['RQD']<25),'Jn']=15
df.loc[((df['RQD']>=25)&(df['RQD']<75)),'Jn']=12
df.loc[((df['RQD']>=75)&(df['RQD']<90)),'Jn']=9
df.loc[((df['RQD']>=90)&(df['RQD']<=100)),'Jn']=6

## Jr_Max, consider from lowest case first and overwrites
### slickensided planar
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('slickensided', case = False))|(df['Details'].str.contains(
    'slickensided',case = False)))&((df['GEOL_DESC'].str.contains('planar', case = False))|(df['Details'].str.contains(
    'planar',case = False)))),'Jr_Max']=0.5
### smooth planar
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('smooth', case = False))|(df['Details'].str.contains(
    'smooth',case = False)))&((df['GEOL_DESC'].str.contains('planar', case = False))|(df['Details'].str.contains(
    'planar',case = False)))),'Jr_Max']=1
### rough planar
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('rough', case = False))|(df['Details'].str.contains(
    'rough',case = False)))&((df['GEOL_DESC'].str.contains('planar', case = False))|(df['Details'].str.contains(
    'planar',case = False)))),'Jr_Max']=1.5
### slickensided undulating
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('slickensided', case = False))|(df['Details'].str.contains(
    'slickensided',case = False)))&((df['GEOL_DESC'].str.contains('undulating', case = False))|(df['Details'].str.contains(
    'undulating',case = False)))),'Jr_Max']=1.5
### smooth undulating
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('smooth', case = False))|(df['Details'].str.contains(
    'smooth',case = False)))&((df['GEOL_DESC'].str.contains('undulating', case = False))|(df['Details'].str.contains(
    'undulating',case = False)))),'Jr_Max']=2
### rough undulating
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('rough', case = False))|(df['Details'].str.contains(
    'rough',case = False)))&((df['GEOL_DESC'].str.contains('undulating', case = False))|(df['Details'].str.contains(
    'undulating',case = False)))),'Jr_Max']=3
### rough/smooth/slickensided stepped
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('stepped', case = False))|(df['Details'].str.contains('stepped',
                                                                                    case = False)))),'Jr_Max']=4

## Jr_Min, consider in reverse order
### rough/smooth/slickensided stepped
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('stepped', case = False))|(df['Details'].str.contains('stepped',
                                                                                    case = False)))),'Jr_Min']=4
### rough undulating
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('rough', case = False))|(df['Details'].str.contains(
    'rough',case = False)))&((df['GEOL_DESC'].str.contains('undulating', case = False))|(df['Details'].str.contains(
    'undulating',case = False)))),'Jr_Min']=3
### smooth undulating
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('smooth', case = False))|(df['Details'].str.contains(
    'smooth',case = False)))&((df['GEOL_DESC'].str.contains('undulating', case = False))|(df['Details'].str.contains(
    'undulating',case = False)))),'Jr_Min']=2
### slickensided undulating
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('slickensided', case = False))|(df['Details'].str.contains(
    'slickensided',case = False)))&((df['GEOL_DESC'].str.contains('undulating', case = False))|(df['Details'].str.contains(
    'undulating',case = False)))),'Jr_Min']=1.5
### rough planar
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('rough', case = False))|(df['Details'].str.contains(
    'rough',case = False)))&((df['GEOL_DESC'].str.contains('planar', case = False))|(df['Details'].str.contains(
    'planar',case = False)))),'Jr_Min']=1.5
### smooth planar
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('smooth', case = False))|(df['Details'].str.contains(
    'smooth',case = False)))&((df['GEOL_DESC'].str.contains('planar', case = False))|(df['Details'].str.contains(
    'planar',case = False)))),'Jr_Min']=1
### slickensided planar
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('slickensided', case = False))|(df['Details'].str.contains(
    'slickensided',case = False)))&((df['GEOL_DESC'].str.contains('planar', case = False))|(df['Details'].str.contains(
    'planar',case = False)))),'Jr_Min']=0.5

## Ja Max: consider from smallest up
### Clean 
df.loc[is_rock&((df['GEOL_DESC'].str.contains('clean', case = False))|(df['Details'].str.contains('clean',case = False))),
       'Ja_Max']=1
### Iron / Manganese Staining
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('iron', case = False))|(df['Details'].str.contains('iron',case = False))|
       (df['GEOL_DESC'].str.contains('manganese',case = False))|(df['Details'].str.contains('manganese',case = False))|
       (df['GEOL_DESC'].str.contains('stain',case = False))|(df['Details'].str.contains('stain',case = False)))),
       'Ja_Max']=1.5
### Sand infill
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('sand', case = False))|(df['Details'].str.contains('sand',case = False)))),
       'Ja_Max']=2
### Clay infill
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('clay', case = False))|(df['Details'].str.contains('clay',case = False))|
                (df['GEOL_DESC'].str.contains('quartz infill',case = False))|(df['Details'].str.contains('quartz infill',
                                                                                                    case = False))
                )),'Ja_Max']=3
### Kaolin / chlorite
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('kaolin', case = False))|(df['Details'].str.contains('kaolin',case = False))|
       (df['GEOL_DESC'].str.contains('chlorite',case = False))|(df['Details'].str.contains('chlorite',case = False))|
                (df['GEOL_DESC'].str.contains('calcite coated',case = False))|(df['Details'].str.contains('calcite coated',
                                                                                                    case = False)))),
       'Ja_Max']=4

## Ja Min: consider from biggest down
### Kaolin / chlorite
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('kaolin', case = False))|(df['Details'].str.contains('kaolin',case = False))|
       (df['GEOL_DESC'].str.contains('chlorite',case = False))|(df['Details'].str.contains('chlorite',case = False)))),
       'Ja_Min']=4
### Clay infill
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('clay', case = False))|(df['Details'].str.contains('clay',case = False)))),
       'Ja_Min']=3
### Sand infill
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('sand', case = False))|(df['Details'].str.contains('sand',case = False))|
                (df['GEOL_DESC'].str.contains('calcite coated',case = False))|(df['Details'].str.contains('calcite coated',
                                                                                                          case = False))|
                (df['GEOL_DESC'].str.contains('quartz infill',case = False))|(df['Details'].str.contains('quartz infill',
                                                                                                         case = False))
                )),'Ja_Min']=2
### Iron / Manganese Staining
df.loc[(is_rock&((df['GEOL_DESC'].str.contains('iron', case = False))|(df['Details'].str.contains('iron',case = False))|
       (df['GEOL_DESC'].str.contains('manganese',case = False))|(df['Details'].str.contains('manganese',case = False))|
       (df['GEOL_DESC'].str.contains('stain',case = False))|(df['Details'].str.contains('stain',case = False)))),
       'Ja_Min']=1.5
### Clean 
df.loc[is_rock&((df['GEOL_DESC'].str.contains('clean', case = False))|(df['Details'].str.contains('clean',case = False))),
       'Ja_Min']=1

#Step 3.4: Calculates Q'
df['Jw']=''
df['SRF']=''
df["Q'"]=((df['RQD']/df['Jn'])*(df['Jr_Min']/df['Ja_Max']))
df['Q']=''

In [9]:
#df.head()
import datetime
time_now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
df.to_excel('concat_processed_{}.xlsx'.format(time_now))

In [15]:
import re


# Wash boring 
WB_list = ['WASHING','Wash boring', 'wash boring', 'Wash-boring', 'wash-boring', 
           'Wash-drilling', 'wash-drilling', 'Wash drilling', 'WASH BORING']

washing = ((df["Details"].str.contains('|'.join(WB_list))) | 
           (df["GEOL_DESC"].str.contains('|'.join(WB_list))))

df.loc[washing, 'Material'] = 'Wash boring'


# Transported vs in situ
transported = df["WETH_GRAD"].isna()
df.loc[(~washing)&transported, 'Material'] = 'Transported'
df.loc[(~washing)&(~transported), 'Material'] = 'In situ'

# Further check if some are simply missing the weathering grade 
# - assume everything down an insitt intervals are in situ (applies for vertical / steep inclined BH)
transported = df['Material']=='Transported'
insitu = df['Material']=='In situ'

for BH in HOLE_ID:
    sel = df['HOLE_ID']==BH
    # the first in situ "From" level
    try:
        BH_insitu_level = df.loc[sel & insitu, 'DEPTH_FROM'].iloc[0]
        insitu_below = df['DEPTH_FROM'] > BH_insitu_level
        df.loc[sel & transported & insitu_below , 'Material'] = 'In situ'
    except IndexError: # Cases without  insitu material in the BH
        pass

# Lithology & Label

# Method 1: Using re.search() with end of string anchor $
def extract_last_bracket1(text):
    try:
        matches = re.findall(r'\((.*?)\)', text)
        return matches[-1] if matches else None
    except TypeError: # Empty description 
        pass
    
def extract_capitals(text):
    # First remove content in brackets
    try:
        text_without_brackets = re.sub(r'\([^)]*\)', '', text)
        # Remove the first character
        text_without_first = text_without_brackets[1:]
        # Find sequences of capital letters
        capitals = re.findall(r'[A-Z]+', text_without_first)
        if len(capitals)==0:
            capitals = ""
        elif len(capitals)==1:
            capitals = capitals[0]
        else:
            capitals = " & ".join(capitals)
        return capitals
    except TypeError: # Empty description 
        pass
    
transported = df['Material']=='Transported'

df.loc[transported, 'LITH'] = df.loc[transported, 'GEOL_DESC'].apply(extract_last_bracket1)
df.loc[transported, 'PRIMARY'] = df.loc[transported, 'GEOL_DESC'].apply(extract_capitals)
df['CAPITALS'] = df['GEOL_DESC'].apply(extract_capitals)

# Wash-bored & In situ material
df.loc[washing, 'LITH'] = 'WASHING'
df.loc[df['Material']=="In situ", 'LITH'] = df.loc[df['Material']=="In situ", 'WETH_GRAD']

df.to_excel('Intervals.xlsx')